In [ ]:
#As always, we import everything
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
def show_hist(data, title, xlabel, ylabel):
    plt.hist(data, range=[0, 40000])
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

In [ ]:
def show_box_plot(data1, data2, legend1, legend2):
    plt.boxplot([data1, data2])
    plt.xticks([1, 2], [legend1, legend2])
    plt.show()

In [ ]:
def show_bar_plots(counts1, total1, counts2, total2, colors, title, ylabel, xlabels, xticklabels):
    width = 1/(len(counts1) + 1)
    fig, ax = plt.subplots(figsize=(20,7))
    pos = list(range(2))

    plt.bar(pos, [counts1[0] / total1, counts2[0] / total2], width, color=colors[0])
    plt.bar([p + width for p in pos], [counts1[1] / total1, counts2[1] / total2], width, color=colors[1])
    if len(counts1) > 2:
        plt.bar([p + 2 * width for p in pos], [counts1[2] / total1, counts2[2] / total2], width, color=colors[2])

    ax.set_ylabel(ylabel)

    ax.set_title(title)

    denom = 1 if len(counts1) == 3 else 2
    ax.set_xticks([p + width/denom for p in pos])

    plt.legend(xlabels, loc='upper left')

    ax.set_xticklabels(xticklabels)

    plt.grid(axis='y')
    plt.show()

### 1. A naive analysis

In [ ]:
data = pd.read_csv('lalonde.csv')
data

In [ ]:
data_non_treatment = data[data.treat == 0][['re78']].values
data_treatment = data[data.treat == 1][['re78']].values

non_treatment_count = len(data_non_treatment)
treatment_count = len(data_treatment)

In [ ]:
show_hist(data_non_treatment, "Histogram of revenue for non-treatment", "Revenue in 1978", "Frequency")

In [ ]:
show_hist(data_non_treatment, "Histogram of revenue for treatment", "Revenue in 1978", "Frequency")

In [ ]:
show_box_plot(data_non_treatment, data_treatment, 'non-treatment', 'treatment')

As we can see from those basic histograms and the box plot, the 'treatment' population has a lower income (except for a few outliers).
A naïve researcher could conclude that the treatment is not only inneficient, but also diminishes the potential income of the participant.

### 2. A closer look at the data

Now, let's look at each variable to see if there is differences between the two groups.

In [ ]:
age_non_treatment = data[data.treat == 0][['age']].values
age_treatment = data[data.treat == 1][['age']].values

show_box_plot(age_non_treatment, age_treatment, 'non-treatment', 'treatment')

First, we can see that the treated group is generally younger.

In [ ]:
educ_non_treatment = data[data.treat == 0][['educ']].values
educ_treatment = data[data.treat == 1][['educ']].values

show_box_plot(educ_non_treatment, educ_treatment, 'non-treatment', 'treatment')

For the education, the two groups are very similar, except for a few outliers.

In [ ]:
black_non_treatment = len(data[(data.treat == 0) & (data.black == 1)])
hispan_non_treatment = len(data[(data.treat == 0) & (data.hispan == 1)])
white_non_treatment = non_treatment_count - black_non_treatment - hispan_non_treatment

black_treatment = len(data[(data.treat == 1) & (data.black == 1)])
hispan_treatment = len(data[(data.treat == 1) & (data.hispan == 1)])
white_treatment = treatment_count - black_treatment - hispan_treatment

In [ ]:
show_bar_plots([black_non_treatment, hispan_non_treatment, white_non_treatment], 
               non_treatment_count, 
               [black_treatment, hispan_treatment, white_treatment],
               treatment_count,
               ['#b2182b','#d6604d','#f0b572'],
               'Race of participants',
               'Race (%)',
               ['Black', 'Hispanic', 'White'],
               ['Non-treatment', 'Treatment'])

We can see here that there is a huge difference in terms of races in the two groups. The majority of the non-treatment is white, whereas the overwhelming majority of the treatment group is black. This can influence the study as the race in the US has a significant correlation with the socio-economic conditions of the person.

In [ ]:
married_non_treatment = len(data[(data.treat == 0) & (data.married == 1)])
not_married_non_treatment = non_treatment_count - married_non_treatment

married_treatment = len(data[(data.treat == 1) & (data.married == 1)])
not_married_treatment = treatment_count - married_treatment

In [ ]:
"""
width = 1/3 
colors = ['#b2182b','#d6604d']
fig, ax = plt.subplots(figsize=(20,7))
pos = list(range(2))

plt.bar(pos, [married_non_treatment / non_treatment_count, married_treatment / treatment_count], width, color=colors[0])
plt.bar([p + width for p in pos], [not_married_non_treatment / non_treatment_count, not_married_treatment / treatment_count], width, color=colors[1])

ax.set_ylabel('Married (%)')

ax.set_title('Marital status of participants')

ax.set_xticks([p + width/2 for p in pos])

plt.legend(['Married', 'Not Married'], loc='upper left')

ax.set_xticklabels(['Non-treatment', 'Treatment'])

plt.grid(axis='y')
plt.show()
"""
show_bar_plots([married_non_treatment, not_married_non_treatment], 
               non_treatment_count, 
               [married_treatment, not_married_treatment],
               treatment_count,
               ['#b2182b','#d6604d'],
               'Marital status of participants',
               'Married (%)',
               ['Married', 'Not Married'],
               ['Non-treatment', 'Treatment'])

There is a much bigger share of the treated group which is not married, which can partly be explained by the fact that the treated group is younger.

In [ ]:
degree_non_treatment = len(data[(data.treat == 0) & (data.nodegree == 0)])
nodegree_non_treatment = non_treatment_count - degree_non_treatment

degree_treatment = len(data[(data.treat == 1) & (data.nodegree == 0)])
nodegree_treatment = treatment_count - degree_treatment

In [ ]:
show_bar_plots([degree_non_treatment, nodegree_non_treatment], 
               non_treatment_count, 
               [degree_treatment, nodegree_treatment],
               treatment_count,
               ['#b2182b','#d6604d'],
               'Share of participants who have a degree',
               'Degree (%)',
               ['Degree', 'No Degree'],
               ['Non-treatment', 'Treatment'])

There is a small difference between the two groups. There is 10% more people in the treatment group who don't have a degree.

In [ ]:
re74_non_treatment = data[data.treat == 0][['re74']].values
re74_treatment = data[data.treat == 1][['re74']].values

show_box_plot(re74_non_treatment, re74_treatment, 'non-treatment', 'treatment')

For the revenue in 1974, a large share of the participants seem to have an income of $0, which seems to indicate that there is missing data.

In [ ]:
re75_non_treatment = data[data.treat == 0][['re75']].values
re75_treatment = data[data.treat == 1][['re75']].values

show_box_plot(re75_non_treatment, re75_treatment, 'non-treatment', 'treatment')

The results are similar to the ones for 1974.

In [ ]:
from sklearn import linear_model
logistic = linear_model.LogisticRegression(max_iter=100, tol=1e-9)

In [ ]:
y = data.treat.values
X = data.drop(['id', 'treat', 're78'], axis=1).values
X

In [ ]:
logistic.fit(X, y)

In [ ]:
probas = logistic.predict_proba(X)

In [ ]:
score_dfs = pd.concat((pd.DataFrame(probas)[1], data.treat), axis=1).sort_values([1], ascending = False)
score_dfs

In [ ]:
matching = []
last_matched = False
i = 0
for index, row in score_dfs.iterrows():
    if i != 0:
        if not last_matched:
            last = score_dfs.iloc[i - 1]
            if row.treat != last.treat:
                #matching[row.index] = last.index
                if row.treat == 0:
                    matching.append((row.name, last.name))
                else:
                    matching.append((last.name, row.name))
                last_matched = True
        else:
            last_matched = False
    i += 1
non_treat_matched = data.loc[pd.DataFrame(matching)[0]]
treat_matched = data.loc[pd.DataFrame(matching)[1]]

In [ ]:
matched_non_treatment = non_treat_matched[['re78']].values
matched_data_treatment = treat_matched[['re78']].values

non_treatment_count = len(matched_non_treatment)
treatment_count = len(matched_data_treatment)

In [ ]:
show_hist(matched_non_treatment, "Histogram of revenue for non-treatment", "Revenue in 1978", "Frequency")

In [ ]:
show_hist(matched_data_treatment, "Histogram of revenue for non-treatment", "Revenue in 1978", "Frequency")

In [ ]:
show_box_plot(matched_non_treatment, matched_data_treatment, 'non-treatment', 'treatment')

In [ ]:
age_non_treatment = non_treat_matched[['age']].values
age_treatment = treat_matched[['age']].values

show_box_plot(age_non_treatment, age_treatment, 'non-treatment', 'treatment')

In [ ]:
educ_non_treatment = non_treat_matched[['educ']].values
educ_treatment = treat_matched[['educ']].values

show_box_plot(educ_non_treatment, educ_treatment, 'non-treatment', 'treatment')

In [ ]:
black_non_treatment = len(non_treat_matched[non_treat_matched.black == 1])
hispan_non_treatment = len(non_treat_matched[non_treat_matched.hispan == 1])
white_non_treatment = non_treatment_count - black_non_treatment - hispan_non_treatment

black_treatment = len(treat_matched[treat_matched.black == 1])
hispan_treatment = len(treat_matched[treat_matched.hispan == 1])
white_treatment = treatment_count - black_treatment - hispan_treatment

In [ ]:
show_bar_plots([black_non_treatment, hispan_non_treatment, white_non_treatment], 
               non_treatment_count, 
               [black_treatment, hispan_treatment, white_treatment],
               treatment_count,
               ['#b2182b','#d6604d','#f0b572'],
               'Race of participants',
               'Race (%)',
               ['Black', 'Hispanic', 'White'],
               ['Non-treatment', 'Treatment'])

In [ ]:
married_non_treatment = len(non_treat_matched[non_treat_matched.married == 1])
not_married_non_treatment = non_treatment_count - married_non_treatment

married_treatment = len(treat_matched[treat_matched.married == 1])
not_married_treatment = treatment_count - married_treatment

In [ ]:
show_bar_plots([married_non_treatment, not_married_non_treatment], 
               non_treatment_count, 
               [married_treatment, not_married_treatment],
               treatment_count,
               ['#b2182b','#d6604d'],
               'Marital status of participants',
               'Married (%)',
               ['Married', 'Not Married'],
               ['Non-treatment', 'Treatment'])

In [ ]:
degree_non_treatment = len(non_treat_matched[non_treat_matched.nodegree == 0])
nodegree_non_treatment = non_treatment_count - degree_non_treatment

degree_treatment = len(treat_matched[treat_matched.nodegree == 0])
nodegree_treatment = treatment_count - degree_treatment

In [ ]:
show_bar_plots([degree_non_treatment, nodegree_non_treatment], 
               non_treatment_count, 
               [degree_treatment, nodegree_treatment],
               treatment_count,
               ['#b2182b','#d6604d'],
               'Share of participants who have a degree',
               'Degree (%)',
               ['Degree', 'No Degree'],
               ['Non-treatment', 'Treatment'])

In [ ]:
re74_non_treatment = non_treat_matched[['re74']].values
re74_treatment = treat_matched[['re74']].values

show_box_plot(re74_non_treatment, re74_treatment, 'non-treatment', 'treatment')

In [ ]:
re75_non_treatment = non_treat_matched[['re75']].values
re75_treatment = treat_matched[['re75']].values

show_box_plot(re75_non_treatment, re75_treatment, 'non-treatment', 'treatment')